In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split

C:\Users\arman\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("Reddit_Data.csv")

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (37249, 2)


,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [8]:
df.dropna(subset=["clean_comment", "category"], inplace=True)  # Drop rows with missing comment text or category
df.drop_duplicates(subset=["clean_comment"], inplace=True) # Remove duplicates
df["clean_comment"] = df["clean_comment"].str.strip() # Remove leading/trailing spaces from comments
print(df["category"].value_counts())

category
 1    15771
 0    12778
-1     8250
Name: count, dtype: int64


In [9]:
df.head(10)

,clean_comment,category,sentiment_label
0,family mormon have never tried explain them th...,1,positive
1,buddhism has very much lot compatible with chr...,1,positive
2,seriously don say thing first all they won get...,-1,negative
3,what you have learned yours and only yours wha...,0,neutral
4,for your own benefit you may want read living ...,1,positive
5,you should all sit down together and watch the...,-1,negative
6,was teens when discovered zen meditation was t...,1,positive
7,jesus was zen meets jew,0,neutral
8,there are two varieties christians dogmatic th...,-1,negative
9,dont worry about trying explain yourself just ...,1,positive


In [5]:
label_map = {
    -1: "negative",
     0: "neutral",
     1: "positive"
}

df["sentiment_label"] = df["category"].map(label_map)
print(df["sentiment_label"].value_counts())

sentiment_label
positive    15771
neutral     12778
negative     8250
Name: count, dtype: int64


### NLTK preprocessing

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [12]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
def preprocess(text):
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
    tokens = word_tokenize(text)  # tokenize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # remove stopwords + lemmatize
    return " ".join(tokens)

df["processed_text"] = df["clean_comment"].apply(preprocess)
df.head()

,clean_comment,category,sentiment_label,processed_text
0,family mormon have never tried explain them th...,1,positive,family mormon never tried explain still stare ...
1,buddhism has very much lot compatible with chr...,1,positive,buddhism much lot compatible christianity espe...
2,seriously don say thing first all they won get...,-1,negative,seriously say thing first get complex explain ...
3,what you have learned yours and only yours wha...,0,neutral,learned want teach different focus goal wrappi...
4,for your own benefit you may want read living ...,1,positive,benefit may want read living buddha living chr...


In [15]:
df[["processed_text", "sentiment_label"]].to_csv("cleaned_dataset.csv", index=False, encoding="utf-8")

In [16]:
df.to_csv("cleaned_dataset.csv", index=False, encoding="utf-8")

print("✅ Cleaned dataset saved as cleaned_dataset.csv")

✅ Cleaned dataset saved as cleaned_dataset.csv
